# Theory

This notebook discusses theoretical aspects of the network.

## Separable Convolutions

The bottleneck and downsampling blocks make use of separable 
convolutions in order to improve performance. A separable convolution
involves breaking a standard CNN-style 2D convolution into two separate
convolutions, one for spatial mixing and one for channel mixing. 
In the literature spatial mixing is often called a depthwise convolution 
while channel mixing is called a pointwise convolution. 
For the rest of this document, subscripts $d$ and $p$ will be used to denote
depthwise and pointwise CNN-style 2D convolutions respectively.

Let

$$
\begin{align*}
    N_o = \text{number of input filters} \\
    N_i = \text{number of output filters} \\
    L_r = \text{input rows} \\
    L_c = \text{input columns} \\
    F_r = \text{filter rows} \\
    F_c = \text{filter columns} \\
\end{align*}
$$

Under a standard CNN-style 2D convolution (assume `padding=same`), we have

$$
\begin{align*}
    \text{weights} = N_o N_i F_r F_c 
    &&
    \text{MACs} = N_o N_i F_r F_c L_r L_c
\end{align*}
$$

By splitting CNN-style 2D convolution into spatial and depthwise components 
we instead have two separate convolutions

$$
\begin{align*}
    \text{weights}_d = N_i F_r F_c
    &&
    \text{weights}_p = N_o N_i 
    \\
    \text{MACs}_d = N_i F_r F_c L_r L_c \\
    &&
    \text{MACs}_p = N_o N_i L_r L_c 
\end{align*}
$$

The total weights and MACs for both convolutions is

$$
\begin{align*}
    \text{weights}_{tot} = N_i (N_o + F_r F_c)
    &&
    \text{MACs}_{tot} = N_i L_r L_c (N_o + F_r F_c)
\end{align*}
$$

Finally, in order to see an improvement in memory / MACs we must 
satisfy the following inequalities 
(which quickly simlify to the same inequality).

For memory:

$$
\begin{align*}
    N_o N_i F_r F_c &\geq N_i (N_o + F_r F_c) \\
    N_o F_r F_c &\geq N_o + F_r F_c \\
    N_o F_r F_c &\geq N_o + F_r F_c
\end{align*}
$$

For MACs:

$$
\begin{align*}
    N_o N_i F_r F_c L_r L_c &\geq N_i L_r L_c (N_o + F_r F_c) \\
    N_o F_r F_c &\geq N_o + F_r F_c \\
    N_o F_r F_c &\geq N_o + F_r F_c
\end{align*}
$$

If we assume a resonable kernel size of $F_r = F_c = 3$ we have

$$
\begin{align*}
    9 N_o &\geq N_o + 9 \\
    N_o &\geq \frac{9}{8} \\
\end{align*}
$$

This confirms that for reasonable parameters we will see a reduction in 
memory / compute with separable convolutions while still mixing information
in both spatial and channel dimensions.

## Generalized Separable Convolutions

The case mentioned above dealt specifically with depthwise convolutions such that
$N_o = N_i$. Now consider the general case of depthwise convolutions 
such that $\gamma$ output feature maps are produced for each input feature map,
i.e. $N_o = \gamma N_i$. We will still permit a $N_o \rightarrow N_i$ mapping in 
the pointwise step, but note that this calculation could be simplified under the 
constraint that the pointwise $N_i = N_o$ ($N_o$ would be expressed in terms of $\gamma$).

Assuming that the output of the depthwise convolution is given as input to 
the pointwise convolution, we have

$$
\begin{align*}
    \text{weights}_d = \gamma N_i F_r F_c
    &&
    \text{weights}_p = \gamma N_o N_i
    \\
    \text{MACs}_d = \gamma N_i F_r F_c L_r L_c \\
    &&
    \text{MACs}_p = \gamma N_o N_i L_r L_c 
\end{align*}
$$

We must have $\gamma \geq 1$, as a reduction in channels with a depthwise 
convolution could only be achieved by throwing away input feature maps.
In light of this, it would appear that multiplying depth at the depthwise
convolution stage will increase the memory and compute requirements of the 
network. 

Instead, let us consider the effect of distributing a desired depth 
multiplication over both the depthwise and pointwise steps. Suppose that the
target depth multiplication is $\gamma$, i.e. $N_o = \gamma N_i$. We will perform
a $\sqrt{\gamma}$ multiplication at the depthwise convolution and a $\sqrt{\gamma}$ 
multiplication at the pointwise convolution.

The depthwise step will produce $\sqrt{\gamma} N_i$ feature maps that will be 
fed as input to the pointwise step. For clarity, the pointwise step will now have

$$
\begin{align*}
    N_i = \sqrt{\gamma} N_i^{\text{depthwise} }
    &&
    N_o = \sqrt{\gamma} N_i = \gamma N_i^{\text{depthwise}}
\end{align*}
$$

This gives 

$$
\begin{align*}
    \text{weights}_d = \sqrt{\gamma} N_i F_r F_c
    &&
    \text{weights}_p = \gamma^\frac{3}{2} N_i^2
    \\
    \text{MACs}_d = \sqrt{\gamma} N_i F_r F_c L_r L_c \\
    &&
    \text{MACs}_p = \gamma^\frac{3}{2} N_i^2 L_r L_c 
\end{align*}
$$

The total weights and MACs for both convolutions is

$$
\begin{align*}
    \text{weights}_{tot} &= \sqrt{\gamma} N_i (N_i \gamma + F_r F_c) \\
    \text{MACs}_{tot} &= \sqrt{\gamma} N_i L_r L_c (N_i \gamma + F_r F_c)
\end{align*}
$$

Recall that for unit depthwise multiplication we had

$$
\begin{align*}
    \text{weights}_{tot} &= N_i (N_o + F_r F_c) \\
    \text{MACs}_{tot} &= N_i L_r L_c (N_o + F_r F_c)
\end{align*}
$$

Under the constraint of $\gamma \geq 1$ it is clear that distributing the depth
multiplication across the depthwise and pointwise steps does not improve
memory or compute. This strategy may be helpful in the tail where a significant
increase in depth is needed over a short number of convolutional layers. 
Rather than force a mapping of $N_i$ input channels to $\gamma N_i$ output 
channels directly, the distributed approach creates $\sqrt{\gamma N_i}$ weak 
features on a per-channel basis which are then mapped to $\gamma N_i$ output 
features.